In [ ]:
# ========================================
# X-Ray Classifier with PyTorch Lightning
# Google Colab Version
# ========================================

%pip install pytorch-lightning torchmetrics albumentations seaborn matplotlib pandas tqdm -q

import os

import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from torchmetrics.classification import Accuracy
import pandas as pd
from sklearn.model_selection import train_test_split

# Upload your dataset to Google Drive first!

DATA_ROOT = "../COVID-19_Radiography_Dataset"  # adjust path if needed
  # adjust to your folder
BATCH_SIZE = 32
EPOCHS = 10
LR = 1e-4

from dataset_covid import CovidXrayDataset, CLASSES
from transforms import get_train_transforms, get_valid_transforms

# ---------------- DATASET ----------------
all_samples = []
for c in CLASSES:
    img_dir = os.path.join(DATA_ROOT, c, "images")
    if not os.path.isdir(img_dir):
        img_dir = os.path.join(DATA_ROOT, c)
    if not os.path.isdir(img_dir):
        continue
    files = [f for f in os.listdir(img_dir) if f.lower().endswith(('.png','.jpg','.jpeg'))]
    for f in files:
        all_samples.append({"path": os.path.join(img_dir, f), "label": c})

df = pd.DataFrame(all_samples)
train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)
train_df, val_df  = train_test_split(train_df, test_size=0.15, stratify=train_df["label"], random_state=42)

train_ds = CovidXrayDataset(train_df, transform=get_train_transforms())
val_ds   = CovidXrayDataset(val_df, transform=get_valid_transforms())
test_ds  = CovidXrayDataset(test_df, transform=get_valid_transforms())

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ---------------- MODEL ----------------
class XrayClassifier(pl.LightningModule):
    def __init__(self, num_classes=len(CLASSES), lr=LR):
        super().__init__()
        self.save_hyperparameters()
        self.model = models.densenet121(weights="IMAGENET1K_V1")
        num_ftrs = self.model.classifier.in_features
        self.model.classifier = nn.Linear(num_ftrs, num_classes)
        self.criterion = nn.CrossEntropyLoss()
        self.acc = Accuracy(task="multiclass", num_classes=num_classes, average="macro")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        outputs = self(imgs)
        loss = self.criterion(outputs, labels)
        acc = self.acc(outputs, labels)
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_acc", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        outputs = self(imgs)
        loss = self.criterion(outputs, labels)
        acc = self.acc(outputs, labels)
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", acc, prog_bar=True)

    def test_step(self, batch, batch_idx):
        imgs, labels = batch
        outputs = self(imgs)
        loss = self.criterion(outputs, labels)
        acc = self.acc(outputs, labels)
        self.log("test_loss", loss, prog_bar=True)
        self.log("test_acc", acc, prog_bar=True)

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=1e-4)

# ---------------- TRAIN ----------------
model = XrayClassifier()

trainer = pl.Trainer(

    max_epochs=EPOCHS,
    accelerator="auto",   # Automatically select GPU if available, else CPU
    devices=1,
    precision=16,
    log_every_n_steps=10
 
    
    
)

trainer.fit(model, train_loader, val_loader)
trainer.test(model, test_loader)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\lightning_fabric\connector.py:571: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:508: You passed `Trainer(accelerator='cpu', precision='16-mixed')` but AMP with fp16 is not supported on CPU. Using `precision='bf16-mixed'` instead.
Using bfloat16 Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:428: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


C:\Users\ADMIN\AppData\Roaming\Python\Python313\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:428: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 0/397 [00:00<?, ?it/s]